In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.moo.unsga3 import UNSGA3
from pymoo.factory import get_crossover, get_mutation, get_sampling, get_reference_directions

from pymoo.optimize import minimize
from pymoo.core.callback import Callback

from pooling_functions import MinConcsProblem, MinNumCntrProblems, calc_pools_conc, calc_pool_conc

This notebook relies on pymoo "0.5.0". To install it, one needs to run:

> pip install pymoo==0.5.0

from the command line.

In [2]:
class MyCallback(Callback):
    def __init__(self) -> None:
        super().__init__()
        self.n_evals = []
        self.hist_F = []

    def notify(self, algorithm):
        self.n_evals.append(algorithm.evaluator.n_eval)
        feas = np.where(algorithm.opt.get("feasible"))[0]
        self.hist_F.append(algorithm.opt.get("F")[feas])

Load data:

In [3]:
datay = pd.read_csv('data/yearly_damages.csv', index_col=0)
datay.head()

,ASM,KHM,COK,FJI,PYF,IDN,KIR,LAO,MYS,MHL,...,MWI,MUS,MOZ,SOM,TZA,ZMB,ZWE,BWA,ZAF,SWZ
0,0.000000e+00,1.174686e+05,61567.633482,0.000000,2066.621255,4.932370e+09,0.0,1.468178e+05,1.363927e+10,0.000000,...,0.0,4.132596e+08,1.240871e+07,1.518563e+07,8.905124e+09,0.0,0.0,0.0,1.297125e+06,0.0
1,0.000000e+00,2.135547e+07,0.000000,0.000000,1817.411519,0.000000e+00,0.0,2.456694e+06,5.648761e+06,81.644624,...,0.0,3.394774e+05,1.486315e+08,0.000000e+00,7.669974e+06,0.0,0.0,0.0,0.000000e+00,0.0
2,2.615973e+07,6.706448e+08,2374.730234,0.000000,42.486543,6.417800e+07,0.0,0.000000e+00,3.386322e+05,0.000000,...,0.0,6.093447e+07,1.342145e+07,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
3,6.236010e+07,3.473528e+07,702363.195433,4825.253640,0.000000,0.000000e+00,0.0,0.000000e+00,2.667397e+09,0.000000,...,0.0,1.005025e+05,1.356082e+04,8.197128e+05,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0
4,0.000000e+00,2.488902e+07,0.000000,158.037936,0.000000,3.155947e+09,0.0,0.000000e+00,0.000000e+00,0.000000,...,0.0,5.339540e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0


Load damages by region:

In [4]:
reg_names = ['EAP', 'LAC', 'SA', 'SSA']

cntrs_reg_pools = []
x_pool_int = []
i = 1

for reg_name in reg_names:
    regs = np.loadtxt('data/{}.txt'.format(reg_name), dtype=str)
    reg_datay = datay[regs.tolist()]

    cntries = np.loadtxt('results/{}_minconc_mincntr.txt'.format(reg_name))
    regs_in_pool = reg_datay.columns[cntries > 0]

    x_pool_int.extend(np.repeat(i, len(regs_in_pool)))
    cntrs_reg_pools.extend(regs_in_pool)
    i += 1

In [5]:
x_fixed_pools = np.array(x_pool_int)

new_cols_ord = cntrs_reg_pools + [col for col in datay.columns.values if not col in cntrs_reg_pools]
datay = datay[new_cols_ord]

num_cntries_to_explore = datay.shape[1]-x_fixed_pools.shape[0]

Define alpha:

In [6]:
RT = 200

alpha = 1-1/RT
alpha

0.995

In [7]:
bools = datay.values >= np.nanquantile(datay.values, alpha, axis=0)

Find optimal global extensions for the regional pools and then find the smallest set of countries in the best pools for each region. The first optimization should be run using parallel processing, for which one would need to uncomment the commented lines below. Also, nm_iter should be increased (25000 was used).

In [11]:
#from pymoo.core.problem import starmap_parallelized_eval
#from multiprocessing.pool import ThreadPool

#n_threads = 4*20
#pool = ThreadPool(n_threads)

N = 4
problem = MinConcsProblem(num_cntries_to_explore, datay, bools, 
                          alpha, calc_pools_conc, N, x_fixed_pools) #,runner=pool.starmap, func_eval=starmap_parallelized_eval)

algorithm = UNSGA3(ref_dirs=get_reference_directions("energy", N, 90),
                    n_offsprings=num_cntries_to_explore,
                    sampling=get_sampling("int_random"),
                    crossover=get_crossover("int_sbx"),
                    mutation=get_mutation("int_pm"),
                    eliminate_duplicates=True)

callback = MyCallback()

nm_iter = 200 #25000

print("FIND OPTIMAL GLOBAL POOLS")
print("Step 1 - Find the pools with the minimum concentration\n")

res = minimize(problem, algorithm, ('n_gen', nm_iter),
                   verbose=False, save_history=False, 
                   callback=callback)

#pool.close()

best_X = res.X[res.F.argmin(0)]

pool_n = 0
for sol in best_X:

    print("Step 2.{} - Find the smallest sub-pool within the best\n{} globally extended pool which maintains the same concentration\n".format(pool_n, reg_names[pool_n]))

    opt_sol_pool = sol == pool_n+1

    x_ = np.hstack([x_fixed_pools, opt_sol_pool*(pool_n+1)])
    cntries = x_ == pool_n+1

    datay_reg = datay.iloc[:, cntries>0]
    bools_reg = bools[:, cntries>0]

    pool_conc = calc_pool_conc(np.repeat(True, datay_reg.shape[1]), datay_reg.values, bools_reg, alpha)[0]

    problem = MinNumCntrProblems(np.sum(opt_sol_pool), datay_reg, bools_reg, alpha,
                                     calc_pools_conc, 1, pool_conc, x_fixed_pools[x_fixed_pools == pool_n+1]*0+1)

    algorithm = GA(pop_size=np.sum(opt_sol_pool),
    sampling=get_sampling("bin_random"),
    crossover=get_crossover("bin_hux"),
    mutation=get_mutation("bin_bitflip"),
    eliminate_duplicates=True)

    res2 = minimize(problem,
                    algorithm,
                    ('n_gen', 200), #750
                    verbose=False,
                    save_history=True,
                    pf=np.array(0.0))

    pool_n += 1

FIND OPTIMAL GLOBAL POOLS
Step 1 - Find the pools with the minimum concentration

Step 2.0 - Find the smallest sub-pool within the best
EAP globally extended pool which maintains the same concentration

Step 2.1 - Find the smallest sub-pool within the best
LAC globally extended pool which maintains the same concentration

Step 2.2 - Find the smallest sub-pool within the best
SA globally extended pool which maintains the same concentration

Step 2.3 - Find the smallest sub-pool within the best
SSA globally extended pool which maintains the same concentration

